In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Trinn 1: Definer Pydantic-modeller for strukturerte resultater

Disse modellene definerer **skjemaet** som agenter vil returnere. Bruk av `response_format` med Pydantic sikrer:
- ✅ Typesikker datauttrekking
- ✅ Automatisk validering
- ✅ Ingen parsingsfeil fra fritekstsvar
- ✅ Enkel betinget ruting basert på felt


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Steg 2: Lag verktøyet for hotellbestilling

Dette verktøyet er det **availability_agent** vil kalle for å sjekke om det er ledige rom. Vi bruker `@ai_function` dekoratøren for å:
- Konvertere en Python-funksjon til et AI-kallbart verktøy
- Automatisk generere JSON-skjema for LLM
- Håndtere parameter-validering
- Aktivere automatisk påkalling av agenter

For denne demoen:
- **Stockholm, Seattle, Tokyo, London, Amsterdam** → Har rom ✅
- **Alle andre byer** → Ingen rom ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Trinn 3: Definer betingelsesfunksjoner for ruting

Disse funksjonene inspiserer agentens svar og bestemmer hvilken vei som skal tas i arbeidsflyten.

**Nøkkelmønster:**
1. Sjekk om meldingen er `AgentExecutorResponse`
2. Analyser den strukturerte utdataen (Pydantic-modell)
3. Returner `True` eller `False` for å kontrollere rutingen

Arbeidsflyten vil evaluere disse betingelsene på **kanten** for å avgjøre hvilken executor som skal kalles neste.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Trinn 4: Opprett Tilpasset Visningsutfører

Utførere er arbeidsflytkomponenter som utfører transformasjoner eller sideeffekter. Vi bruker `@executor`-dekoratøren for å lage en tilpasset utfører som viser det endelige resultatet.

**Nøkkelkonsepter:**
- `@executor(id="...")` - Registrerer en funksjon som en arbeidsflytutfører
- `WorkflowContext[Never, str]` - Typehint for input/output
- `ctx.yield_output(...)` - Leverer det endelige arbeidsflytresultatet


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Trinn 5: Last inn miljøvariabler

Konfigurer LLM-klienten. Dette eksemplet fungerer med:
- **GitHub-modeller** (gratisnivå med GitHub-token)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Trinn 6: Opprett AI-agenter med strukturerte utdata

Vi oppretter **tre spesialiserte agenter**, hver pakket inn i en `AgentExecutor`:

1. **availability_agent** - Sjekker hotelltilgjengelighet ved hjelp av verktøyet
2. **alternative_agent** - Foreslår alternative byer (når det ikke er ledige rom)
3. **booking_agent** - Oppmuntrer til bestilling (når det er ledige rom)

**Nøkkelfunksjoner:**
- `tools=[hotel_booking]` - Gir verktøyet til agenten
- `response_format=PydanticModel` - Tvinger strukturert JSON-utdata
- `AgentExecutor(..., id="...")` - Packer agenten for arbeidsflytbruk


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Trinn 7: Bygg arbeidsflyten med betingede kanter

Nå bruker vi `WorkflowBuilder` til å konstruere grafen med betinget ruting:

**Arbeidsflytstruktur:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Nøkkelmetoder:**
- `.set_start_executor(...)` - Setter startpunktet
- `.add_edge(from, to, condition=...)` - Legger til betinget kant
- `.build()` - Fullfører arbeidsflyten


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Step 8: Kjør Test Case 1 - By UTEN Tilgjengelighet (Paris)

La oss teste **ingen tilgjengelighet**-banen ved å forespørre hoteller i Paris (som ikke har rom i vår simulering).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Step 9: Kjør testtilfelle 2 - By MED tilgjengelighet (Stockholm)

La oss nå teste **tilgjengelighets**-veien ved å be om hoteller i Stockholm (som har rom i vår simulering).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Viktige punkter og neste steg

### ✅ Det du har lært:

1. **WorkflowBuilder-mønster**
   - Bruk `.set_start_executor()` for å definere inngangspunktet
   - Bruk `.add_edge(from, to, condition=...)` for betinget ruting
   - Kall `.build()` for å ferdigstille arbeidsflyten

2. **Betinget ruting**
   - Betingelsesfunksjoner inspiserer `AgentExecutorResponse`
   - Analyser strukturerte utdata for å ta rutebeslutninger
   - Returner `True` for å aktivere en kant, `False` for å hoppe over den

3. **Verktøyintegrasjon**
   - Bruk `@ai_function` for å gjøre Python-funksjoner om til AI-verktøy
   - Agenter kaller verktøy automatisk når nødvendig
   - Verktøy returnerer JSON som agenter kan analysere

4. **Strukturerte utdata**
   - Bruk Pydantic-modeller for typesikker datauttrekking
   - Sett `response_format=MyModel` ved oppretting av agenter
   - Analyser svar med `Model.model_validate_json()`

5. **Egendefinerte executor**
   - Bruk `@executor(id="...")` for å lage arbeidsflyt-komponenter
   - Executorer kan transformere data eller utføre sideeffekter
   - Bruk `ctx.yield_output()` for å produsere arbeidsflytresultater

### 🚀 Virkelige bruksområder:

- **Reisebestilling**: Sjekk tilgjengelighet, foreslå alternativer, sammenlign valg
- **Kundeservice**: Rut basert på sakstype, stemning, prioritet
- **E-handel**: Sjekk lagerstatus, foreslå alternativer, behandle ordre
- **Innholdsmoderering**: Rut basert på toksisitetsnivåer, brukerflagg
- **Godkjenningsarbeidsflyter**: Rut basert på beløp, brukerrolle, risikonivå
- **Flertrinnsprosessering**: Rut basert på datakvalitet, fullstendighet

### 📚 Neste steg:

- Legg til mer komplekse betingelser (flere kriterier)
- Implementer løkker med arbeidsflyt-tilstandsadministrasjon
- Legg til del-arbeidsflyter for gjenbrukbare komponenter
- Integrer med ekte API-er (hotellbestilling, lagersystemer)
- Legg til feilhåndtering og reserveveier
- Visualiser arbeidsflyter med innebygde visualiseringsverktøyene


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Ansvarsfraskrivelse**:
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi streber etter nøyaktighet, vennligst vær oppmerksom på at automatiske oversettelser kan inneholde feil eller unøyaktigheter. Det opprinnelige dokumentet på originalspråket bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for misforståelser eller feiltolkninger som oppstår som følge av bruk av denne oversettelsen.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
